<a href="https://colab.research.google.com/github/jchou03/Natural-Language-Processing/blob/main/PA6/PA6_part_1_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART 1: LSTM FOR LANGUAGE MODELING**

**Task 1:** Follow the tutorial in [here](https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/) to train a word-level LSTM language model. Train the language model on texts
from the file prideAndPrejudice.txt. Before using it to train the language model, you need to first sentence-segment, then
tokenize, then lower case each line of the file using Spacy. Append start-of-sentence token ’\<s\>’ and end-of-sentence
’\</s\>’ token to each sentence and put each sentence in its own line. Use only words that appear more than once in this
corpus and assign UNK tokens for the rest; you may also need to pad sentences that are shorter than 5 (see [here](https://github.com/gabrielloye/LSTM_Sentiment-Analysis/blob/master/main.ipynb) in cell
10-12 for adding unknown: UNK token and padding: PAD token to your vocabulary). Train the language model and
save the trained model (see [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html)). Generate 10 examples of text from it, starting from ’\<s\>’ token and ending at ’\</s\>’
token.

In [ ]:
# load by line

lines = open('prideAndPrejudice.txt').read().splitlines()

print(lines[:10])

['It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.', 'However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.', '"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"', 'Mr. Bennet replied that he had not.', '"But it is," returned she; "for Mrs. Long has just been here, and she told me all about it."', 'Mr. Bennet made no answer.', '"Do you not want to know who has taken it?" cried his wife impatiently.', '"_You_ want to tell me, and I have no objection to hearing it."', 'This was invitation enough.', '"Why, my dear, you must know, Mrs. Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chais

**Checkpoint 1:**

Load texts from by lines. The samples of your loaded text lines should look like the following.

['"It is evident by this," added Jane, "that he comes back no more this winter."',
 '"I do not mean to say that a woman may not be settled too near her family. The far and the near must be relative, and depend on many varying circumstances. Where there is fortune to make the expenses of travelling unimportant, distance becomes no evil. But that is not the case _here_. Mr. and Mrs. Collins have a comfortable income, but not such a one as will allow of frequent journeys--and I am persuaded my friend would not call herself _near_ her family under less than _half_ the present distance."',
 '"Just as you please."',
 '"I remember, when we first knew her in Hertfordshire, how amazed we all were to find that she was a reputed beauty; and I particularly recollect your saying one night, after they had been dining at Netherfield, \'_She_ a beauty!--I should as soon call her mother a wit.\' But afterwards she seemed to improve on you, and I believe you thought her rather pretty at one time."',
 'E

**Checkpoint 2:**

Preprocess your loaded texts with spacy, also convert words into lower-case, and also pad sentences with \<s\> and \</s\>. The preprocessed lines should look like the following.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipe("parser")
nlp.add_pipe("sentencizer")

In [ ]:
def tokenize_sent(sent):
  doc = nlp(sent)
  tokens = ['<s>']
  for token in doc:
    tokens.append(token.text.lower())
  tokens.append('</s>')
  return " ".join(tokens)

for i,line in enumerate(lines):
  lines[i] = tokenize_sent(line)

print(lines[:10])

['<s> it is a truth universally acknowledged , that a single man in possession of a good fortune , must be in want of a wife . </s>', '<s> however little known the feelings or views of such a man may be on his first entering a neighbourhood , this truth is so well fixed in the minds of the surrounding families , that he is considered the rightful property of some one or other of their daughters . </s>', '<s> " my dear mr. bennet , " said his lady to him one day , " have you heard that netherfield park is let at last ? " </s>', '<s> mr. bennet replied that he had not . </s>', '<s> " but it is , " returned she ; " for mrs. long has just been here , and she told me all about it . " </s>', '<s> mr. bennet made no answer . </s>', '<s> " do you not want to know who has taken it ? " cried his wife impatiently . </s>', '<s> " _ you _ want to tell me , and i have no objection to hearing it . " </s>', '<s> this was invitation enough . </s>', '<s> " why , my dear , you must know , mrs. long says 

['<s> it is a truth universally acknowledged , that a single man in possession of a good fortune , must be in want of a wife . </s>', '<s> however little known the feelings or views of such a man may be on his first entering a neighbourhood , this truth is so well fixed in the minds of the surrounding families , that he is considered the rightful property of some one or other of their daughters . </s>', '<s> " my dear mr. bennet , " said his lady to him one day , " have you heard that netherfield park is let at last ? " </s>', '<s> mr. bennet replied that he had not . </s>', '<s> " but it is , " returned she ; " for mrs. long has just been here , and she told me all about it . " </s>', '<s> mr. bennet made no answer . </s>', '<s> " do you not want to know who has taken it ? " cried his wife impatiently . </s>', '<s> " _ you _ want to tell me , and i have no objection to hearing it . " </s>', '<s> this was invitation enough . </s>', '<s> " why , my dear , you must know , mrs. long says 

**Checkpoint 3:**

Define a function to prepare fixed-length sequences from our dataset. Here we specified the length of the sequence as five. You may refer to the tutorial to see how this can be done. Remember to deal with the case of tokens of length less or equal to 5. The following example show what the fixed-length sequence should look like given  input texts. Also compare the size of the processed sequences to see if the number is close.

NOTE: the example is just the processed result of one line, your result sequences should contain processed results for all lines.

In [26]:
# create sequences of length 5 tokens
def create_seq(text, seq_len = 5):
    sequences = []
    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
      for i in range(seq_len, len(text.split())):
        # select sequence of tokens
        seq = text.split()[i-seq_len:i+1]
        # add to the list
        sequences.append(" ".join(seq))

      return sequences
    # if the number of tokens in 'text' is less than 5, add padding
    elif len(text.split()) < seq_len:
      sequence = text.split()
      for i in range(seq_len - len(text.split())):
        sequence.append("</s>")
    # if the number of tokens is exactly 5, then return as is
    else:
      return [text]

def create_seqs (lines):
  sequences = []
  for line in lines:
    sequences.extend(create_seq(line))
  return sequences

sequences = create_seqs(lines)
print(sequences[:10])
print(len(sequences))

['<s> it is a truth universally', 'it is a truth universally acknowledged', 'is a truth universally acknowledged ,', 'a truth universally acknowledged , that', 'truth universally acknowledged , that a', 'universally acknowledged , that a single', 'acknowledged , that a single man', ', that a single man in', 'that a single man in possession', 'a single man in possession of']
138400


<s> " oh ! " said lydia stoutly , " i am not afraid ; for though i _ am _ the youngest , i 'm the tallest . " </s>
['<s> " oh ! " said', '" oh ! " said lydia', 'oh ! " said lydia stoutly', '! " said lydia stoutly ,', '" said lydia stoutly , "', 'said lydia stoutly , " i', 'lydia stoutly , " i am', 'stoutly , " i am not', ', " i am not afraid', '" i am not afraid ;', 'i am not afraid ; for', 'am not afraid ; for though', 'not afraid ; for though i', 'afraid ; for though i _', '; for though i _ am', 'for though i _ am _', 'though i _ am _ the', 'i _ am _ the youngest', '_ am _ the youngest ,', 'am _ the youngest , i', "_ the youngest , i 'm", "the youngest , i 'm the", "youngest , i 'm the tallest", ", i 'm the tallest .", 'i \'m the tallest . "', '\'m the tallest . " </s>']
lines size:  6004
sequences size:  126698


**Checkpoint 4:**

Create inputs and targets (x and y) for training; a sample of your x and y should look like the following. Remember to pad sequences with less than 5 tokens. (NOTE: the sample is just a pair of x,y, not all x's /y's for inputs and targets).

sample x:  " for mrs. long has
sample y:  for mrs. long has just


**Checkpoint 5:**

Compute a list of all words that need to be assigned as UNK. Part of the unknown list should look like the following.

['unwearying', 'defended', 'pointedly', 'although', 'utterly', 'resisted', 'gratification', 'chimney', 'digressions', 'improvements', 'retail', 'imitations', 'china', 'broad', 'faced', 'stuffy', 'breathing', 'port', 'commonest', 'dullest', 'topic', 'speaker', 'rivals', 'muffin', 'life--', 'compliance', 'engrossing', 'talker', 'bets', 'prizes', 'demands', 'hesitating', 'anywhere', 'estimated', 'imposing', 'proclaim', 'breathed', 'truest', 'verily', 'topics', 'military', 'bequeathed', 'gift', 'godfather', 'heavens', 'disregarded', 'legal', 'redress', 'informality', 'bequest', 'conditional', 'forfeited', 'disgraced', 'irritated', 'inhumanity', 'boasting', 'implacability', 'resentments', 'amiable"--but', 'childhood', 'inmates', 'parental', 'confidential', 'superintendence', 'providing', 'dishonest', 'dishonesty', 'impulses', 'degenerate', 'popular', 'guardian', 'superintends', 'trials', 'reverting', 'conversible', 'equals', 'chances', 'shillings', 'regarding', 'misleads', 'arrogant', 'dict

**Checkpoint 6:**

Build two dictionaries (vocabulary), one maps a unique integer to a unique token. The other one is a mapping from the reversed direction. The following is an example of the key-value pairs of the two dictionaries. Your dictionaries should have similar formats.

key: 'the' value:  1502
key: 1327, value:  the
size of the dictionaries (vocabulary): 3907 and 3907


**Checkpoint 7:**

Add unknown and padding to the two dictionaries (check). The size of them should increase by 2.

vocabulary sizes: 3909,3909
token2int['_PAD'] 3908
token2int['_UNK'] 3907


**Checkpoint 8:**

Convert your sequences of tokens into integers, with the dictionaries we just built. Your x's and y's should look like the following.

x in integers: 
[[2514 3142 1934 1341 3284]
 [3142 1934 1341 3284 2482]
 [1934 1341 3284 2482  645]
 ...
 [1002 1502 1824  148 3907]
 [1502 1824  148 3907  533]
 [1824  148 3907  533  322]]

y in integers: 
[[3142 1934 1341 3284 2482]
 [1934 1341 3284 2482  645]
 [1341 3284 2482  645 3167]
 ...
 [1502 1824  148 3907  533]
 [1824  148 3907  533  322]
 [ 148 3907  533  322 1356]]


**Checkpoint 9:**

Build your LSTM model with Pytorch. Your LSTM structure should look like the following:



WordLSTM(
  (emb_layer): Embedding(3909, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=3909, bias=True)
)


**Checkpoint 10:**

Train your model with the processed data.

**Checkpoint 11:**

Generate 10 examples of text from the model, starting from ’\<s\>’ token and ending at ’\</s\>’ token. The generated texts should look more or less natural, instead of being in a very fixed pattern or randomly generated. Some examples are shown below.

<s> " i have been in the world . ' </s>
<s> she was not to have been a very agreeable man , and she could have been in a _UNK manner , " cried elizabeth . ' </s>
<s> i do assure her to the other side , and _UNK her education ; and , in the _UNK . ' i am afraid he had done , and the _UNK , i believe , to be _UNK . ' i have been so well concealed ; for she had not been in the least unwilling to make a very _UNK eye ; but i have been in the least altered by the _UNK , i believe you are to see him at longbourn ; but , and , and the two sisters , " she added : ' i hope i should not have been so _UNK . </s>
<s> " oh ; " and i do , " said she ; but , and she had no idea , and she had no more _UNK to the _UNK of his friend , and the others are not _UNK . </s>
<s> i have not the honour , and she had not seen them . ' </s>
<s> i do , " replied her father , and the others are to be done , but i am afraid i have been so fortunate in love with you . </s>
<s> i do , " cried jane ; but i have not see

**Task 2:**

Compute and report the perplexity of the saved model on test_1.txt and test_2.txt files. Note that the test files are already preprocessed. Compute the complexity of the test_2.txt by saving the model first and then loading it.

**Checkpoint 1:**

Load your test_1.txt file in lines as you did in task 1. The results should look similar to the following.

['<s> the family of dashwood had long been settled in sussex . </s>', '<s> their estate was large , and their residence was at norland park , in the centre of their property , where , for many generations , they had lived in so respectable a manner as to engage the general good opinion of their surrounding acquaintance . </s>', '<s> the late owner of this estate was a single man , who lived to a very advanced age , and who for many years of his life , had a constant companion and housekeeper in his sister . </s>', '<s> but her death , which happened ten years before his own , produced a great alteration in his home ; for to supply her loss , he invited and received into his house the family of his nephew mr. henry dashwood , the legal inheritor of the norland estate , and the person to whom he intended to bequeath it . </s>', "<s> in the society of his nephew and niece , and their children , the old gentleman 's days were comfortably spent . </s>", '<s> his attachment to them all incre

**Checkpoint 2:**

Compute the perplexity on test_1.txt. Your final perplexity should look like the one below. (You may need to implement the perplexity calculation on your own instead of using any existing ones from libraries.)

perplexity on test_1.txt is:  172.75867345155558


**Checkpoint 3:**

Save your model with the 'state_dict()' method. Load your saved model and sample some generated texts to see if it works properly. The sample texts could look like the following one.

'<s> " oh , yes ! pray be _UNK , i am afraid of her , she could not be insensible to the latter method , " said elizabeth ; and she had no reason , i am sure i have not a little mistake , and the _UNK _UNK of his _UNK , she could not help cautioning her to see her ; for she was not in a very different _UNK . " </s>'

**Checkpoint 4:**

Load your test_2.txt file in lines. Compute the perplexity with your loaded model. The perplexity should look close to (or better than) the following one.

perplexity on test_2.txt is:  951.2221688812576


**Task 3:**

Train the language model as before, but with input sequence lengths of 25 (currently, its inputs are of length 5).
You may need to pad some of the shorter sentences to length 25. Save your trained model. Generate 10 examples of text
from it, starting from ’\<s\>’ token and ending at ’\</s\>’ token. **Answer in a separate cell:** Are there differences from the generated examples from
2.1? 

**Task 4:**
Compute and report the perplexity of this saved model on test 1.txt file. **Answer in a separate cell:** Is there any reason why this model is better/worse than the previous one (length = 5)?

NOTE: at this point you should be able to finish the tasks without the checkpoints, try to re-use the code you've implemented.

**Task 5:**

Train the better language model as before but start with pre-trained Glove6B 100d embeddings (see [here](https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76) on how
to incorporate pretrained embeddings in your LSTM model). This time, use all your words in the corpus as vocabulary,
even those occurring only once in the corpus. Only assign UNK token to words that are not in the Glove vocabulary and
initialize random vectors in the embedding matrix for the UNK, \<s\>, \</s\>, and PAD tokens using a Gaussian
distribution with mean 0.0 and σ set to 0.6 (see numpy.random.normal). Save your trained model. Generate 10 examples of text from it,
starting from the ’\<s\>’ token and ending at the ’\</s\>’ token. Are there differences from the generated examples from before?

**Task 6:**
Compute and report the perplexity of this saved model on the test 1.txt file.

NOTE: 

1. The biggest difference between this task and the previous starts from the word-integer dictionary. In addition to the mappings between word and integer, you also need to have a dictionary that maps the word to the word vector in Glove. Don't forget to add UNK, PAD, \<s\> and \</s\> to the dictionary. As these tokens do not exist in Glove, you need np.random.normal(...) to assign them random values.
2. The next difference is since the most of the word embeddings are pre-trained, you need to initialize nn.Embed with the Glove embedding matrix. You also need to adjust the model to fit the embedding as well.
3. The training might take a very long time, make sure you have your GPU utilized for Pytorch and get enough time running it.